# Parameter File

The parameter file is the central control object for a SeisFlows3 workflow. Here we take a look at the anatomy of a parameter file. Parameter files in SeisFlows3 are formatted in the [YAML format (YAML Ain't Markup Language)](https://pyyaml.org/wiki/PyYAMLDocumentation).

## Template

Each workflow starts with the module-only template parameter file which defines the core modules which make up the package. Running `seisflows setup` from the command line will create this file. 

In [4]:
! seisflows setup

creating parameter file: parameters.yaml


In [7]:
! cat parameters.yaml

# //////////////////////////////////////////////////////////////////////////////
#
#                        SeisFlows3 YAML Parameter File
#
# //////////////////////////////////////////////////////////////////////////////
#
# Modules correspond to the structure of the source code, and determine
# SeisFlows3' behavior at runtime. Each module requires its own sub-parameters.
#
# .. rubric::
#   - To determine available options for modules listed below, run:
#       > seisflows print modules
#   - To auto-fill with docstrings and default values (recommended), run:
#       > seisflows configure
#   - To set values as NoneType, use: null
#   - To set values as infinity, use: inf
#
#                                    MODULES
#                                    ///////
# WORKFLOW (str):    The method for running SeisFlows3; equivalent to main()
# SOLVER (str):      External numerical solver to use for waveform simulations
# SYSTEM (str):      Computer architecture of the system being used
#

### How do I choose my modules?

As seen above, each of the modules comes with a default value. But you may want to run a migration, not an inversion. Or run with SPECFEM3D not 2D. As stated in the comments at the top of the file, the `seisflows print modules` command lists out all available options. Don't see an option that works for you? Learn to extend the SeisFlows3 package here: **!!! docs page link here !!!**

In [6]:
! seisflows print modules

                               SEISFLOWS3 MODULES                               
                               //////////////////                               
'+': package, '-': module, '*': class

+ SYSTEM
    - seisflows3
        * base
        * cluster
        * lsf
        * slurm
        * workstation
    - seisflows3-super
        * chinook
        * maui
+ PREPROCESS
    - seisflows3
        * base
        * pyatoa
    - seisflows3-super
        * pyatoa_nz
+ SOLVER
    - seisflows3
        * base
        * specfem2d
        * specfem3d
        * specfem3d_globe
    - seisflows3-super
        * specfem3d_maui
+ POSTPROCESS
    - seisflows3
        * base
    - seisflows3-super
+ OPTIMIZE
    - seisflows3
        * LBFGS
        * NLCG
        * base
    - seisflows3-super
+ WORKFLOW
    - seisflows3
        * base
        * inversion
        * migration
        * test
    - seisflows3-super
        * thrifty_inversion
        * thrifty_maui


### How do I change modules?

Feel free to use any old text editor to edit the YAML file, or you can use the `seisflows par` command to make changes directly from the command line. For example, say we want to use SPECFEM3D

In [8]:
# Changes the current parameter to the given value
! seisflows par solver specfem3d

SOLVER: specfem2d -> specfem3d


In [9]:
# Prints out the current parameter value
! seisflows par solver

SOLVER: specfem3d


### How do I get to a full parameter file?

The module-only parameter file serves as as a template for dynamically generating a full parameter file. Since each module requires it's own unique set of parameters and paths, each parameter file will look different. We can use the `seisflows configure` command to complete our parmater file, based on the chosen modules.

In [10]:
! seisflows configure

filling parameters.yaml w/ default values


## Anatomy of the parameter file

As we will see below, the parameter file has now been generated. Each module will define its own section, separated by a header of comments. Within each header, parameter names, types and descriptions are listed. At the bottom of the parameter file, there is a section defining paths required by the workflow. Section headers will look something: 

In [ ]:
# =============================================================================
#                                    MODULE
#                                    //////                                    
# PARAMETER_NAME (type):
#   Description
# ...
# =============================================================================
PARAMETER_NAME: parameter_value

In [13]:
! head -80 parameters.yaml

# //////////////////////////////////////////////////////////////////////////////
#
#                        SeisFlows3 YAML Parameter File
#
# //////////////////////////////////////////////////////////////////////////////
#
# Modules correspond to the structure of the source code, and determine
# SeisFlows3' behavior at runtime. Each module requires its own sub-parameters.
#
# .. rubric::
#   - To determine available options for modules listed below, run:
#       > seisflows print modules
#   - To auto-fill with docstrings and default values (recommended), run:
#       > seisflows configure
#   - To set values as NoneType, use: null
#   - To set values as infinity, use: inf
#
#                                    MODULES
#                                    ///////
# WORKFLOW (str):    The method for running SeisFlows3; equivalent to main()
# SOLVER (str):      External numerical solver to use for waveform simulations
# SYSTEM (str):      Computer architecture of the system being used
#

In [27]:
! tail --lines=54 parameters.yaml

# =============================================================================
#                                     PATHS                                    
#                                     /////                                    
# SCRATCH:
#   scratch path to hold temporary data during workflow
# OUTPUT:
#   directory to save workflow outputs to disk
# SYSTEM:
#   scratch path to hold any system related data
# LOCAL:
#   path to local data to be used during workflow
# LOGFILE:
#   the main output log file where all processes will track their status
# SOLVER:
#   scratch path to hold solver working directories
# SPECFEM_BIN:
#   path to the SPECFEM binary executables
# SPECFEM_DATA:
#   path to the SPECFEM DATA/ directory containing the 'Par_file', 'STATIONS'
#   file and 'CMTSOLUTION' files
# DATA:
#   path to data available to workflow
# MASK:
#   Directory to mask files for gradient masking
# OPTIMIZE:
#   scratch path to store data related to nonlinear optimization
# MODE

### How do I know what parameters need to be set?

> **NOTE**: Required parameters that can not be set to default values will be listed as `!!! REQUIRED PARAMETER !!!`

We can check the required paths and parameters manually by scrolling through the parameter file, or we can use the `seisflows par --required` command to list them out all at once.

In [29]:
! seisflows par --required

!!! REQUIRED PARAMETER !!!
	MATERIALS
	DENSITY
	ATTENUATION
	NT
	DT
	FORMAT
	CASE
	END
!!! REQUIRED PATH !!!
	SPECFEM_BIN
	SPECFEM_DATA
	MODEL_INIT


## Checking parameter validity

You might be asking, how do I know if my parameters are set correctly? SeisFlows3 modules feature check() functions which dictate correct parameter values. You can run `seisflows init` to run these check() functions. Because we have required parameters still left unset in our parameter file, we expect the `seisflows init` function to throw an error.

In [30]:
! seisflows init

                           PARAMETER FILE READ ERROR                            
                           /////////////////////////                            
Please check that your parameter file is properly formatted in the YAML format.
If you have just run 'seisflows configure', you may have some required
parameters that will need to be filled out before you can proceed. The error
message is:

could not determine a constructor for the tag 'tag:yaml.org,2002:!'
  in "parameters.yaml", line 147, column 12


Let's set some random variables for the required parameters with the `seisflows par` command and try again.

In [31]:
! seisflows par materials elastic
! seisflows par density constant
! seisflows par attenuation False
! seisflows par nt 100
! seisflows par dt .05
! seisflows par format ascii
! seisflows par case data
! seisflows par end 1
! seisflows par specfem_bin ./
! seisflows par specfem_data ./
! seisflows par model_init ./

MATERIALS: !!! REQUIRED PARAMETER !!! -> elastic
DENSITY: !!! REQUIRED PARAMETER !!! -> constant
ATTENUATION: !!! REQUIRED PARAMETER !!! -> False
NT: !!! REQUIRED PARAMETER !!! -> 100
DT: !!! REQUIRED PARAMETER !!! -> .05
FORMAT: !!! REQUIRED PARAMETER !!! -> ascii
CASE: !!! REQUIRED PARAMETER !!! -> data
END: !!! REQUIRED PARAMETER !!! -> 1
SPECFEM_BIN: !!! REQUIRED PATH !!! -> ./
SPECFEM_DATA: !!! REQUIRED PATH !!! -> ./
MODEL_INIT: !!! REQUIRED PATH !!! -> ./


In [32]:
! seisflows init

instantiating SeisFlows3 working state in directory: output


Of course we knew that the above parameters were acceptable. But what if we input an unacceptable parameter into the  parameter file and try again?

In [33]:
! rm -r output/
! seisflows par materials visibily_incorrect_value
! seisflows init

MATERIALS: elastic -> visibily_incorrect_value
                               MODULE CHECK ERROR                               
                               //////////////////                               
seisflows.config module check failed with:

solver: MATERIALS must be in ['ELASTIC', 'ACOUSTIC', 'ISOTROPIC', 'ANISOTROPIC']


And voila, the module check has thrown an error, and told us (the User) how to properly set the value of the materials parameter. Hopefully a combination of thorough explanations in the parameter file section headers, and error catching with `seisflows init` makes crafting your own parameter file a smooth process.

In [37]:
! head -155 parameters.yaml | tail --lines=38


# =============================================================================
#                                    SOLVER                                    
#                                    //////                                    
# MATERIALS (str):
#   Material parameters used to define model. Available: ['ELASTIC': Vp, Vs,
#   'ACOUSTIC': Vp, 'ISOTROPIC', 'ANISOTROPIC']
# DENSITY (str):
#   How to treat density during inversion. Available: ['CONSTANT': Do not
#   update density, 'VARIABLE': Update density]
# ATTENUATION (str):
#   If True, turn on attenuation during forward simulations, otherwise set
#   attenuation off. Attenuation is always off for adjoint simulations.
# COMPONENTS (str):
#   Components used to generate data, formatted as a single string, e.g. ZNE
#   or NZ or E
# SOLVERIO (int):
#   The format external solver files. Available: ['fortran_binary', 'adios']
# NT (float):
#   Number of time steps set in the SPECFEM Par_file
# DT (float):
#   Time step or del

In [38]:
! rm parameters.yaml  # to delete the created file from this working directory